In [1]:
import pandas as pd
import numpy as np
import re
import spacy
from nltk.corpus import stopwords

from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
   
    PER,
    Doc
)

from transformers import AutoTokenizer, AutoModel
import torch

from typing import List, Union
import pickle

import json

d:\Hackathons\Hack_26_04_2024\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [33]:
class Model:
    def __init__(self, model_object = "", model_positive = "", model_relevant =""):


        self.segmenter = Segmenter()
        self.morph_vocab = MorphVocab()
        self.emb = NewsEmbedding()
        self.morph_tagger = NewsMorphTagger(self.emb)
        self.syntax_parser = NewsSyntaxParser(self.emb)

        self.tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_mt_nlu_ru", cache_dir = "model_cache")
        self.model = AutoModel.from_pretrained("ai-forever/sbert_large_mt_nlu_ru", cache_dir = "model_cache")
        
        with open(model_object, "rb") as f:
            self.knn_object = pickle.load(f)

        with open(model_positive, "rb") as f:
            self.knn_positive = pickle.load(f)

        with open(model_relevant, "rb") as f:
            self.knn_relevant = pickle.load(f)

    def __natasha_lemmant(self, input_str):

        doc = Doc(input_str)
        doc.segment(self.segmenter)
        doc.tag_morph(self.morph_tagger)
        doc.parse_syntax(self.syntax_parser)
        for token in doc.tokens:
            token.lemmatize(self.morph_vocab)

        result = ' '.join([_.lemma for _ in doc.tokens])
        return result



    def __clear_string(self, s, stop_words): # Функция для очистки
        # print(re.sub(r'[^а-яА-ЯёЁa-zA-Z]', ' ', str(s).lower()).split())
        s = ' '.join([val for val in re.sub(r'[^а-яА-ЯёЁa-zA-Z]', ' ', str(s).lower()).split() if not val in stop_words])
        # s = ' '.join([val for val in re.sub(r'[^а-яА-ЯёЁ]', ' ', str(s).lower()).split() if not val in stop_words])
        return s

    def __preprocessing(self, answer):
        stop_words = list(stopwords.words('russian'))
        
        text_list = []

        for i in range(len(answer)):
            txt = answer[i]['question_2'] + " " + answer[i]['question_3'] + " " + answer[i]['question_4'] + " " + answer[i]['question_5']
            txt = self.__clear_string(txt, stop_words)
            txt = self.__natasha_lemmant(txt)
            text_list.append(txt)

        return text_list

    def __mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        return sum_embeddings / sum_mask

    def __get_embedding(self, text_list):
        # print(text_list)

        encoded_input = self.tokenizer(text_list, padding=True, truncation=True, max_length=24, return_tensors='pt')
        with torch.no_grad():
            #Compute token embeddings
            model_output = self.model(**encoded_input)

        #Perform pooling. In this case, mean pooling
        # print(encoded_input['attention_mask'])
        # print(model_output)
        embeddings = self.__mean_pooling(model_output, encoded_input['attention_mask']).cpu().numpy()

        return embeddings

    def predict(self, answers: list) -> List[int]:
        
        # Очищаем текст
        answers = self.__preprocessing(answers)

        embeddings = self.__get_embedding(answers)
        
        object_predict = self.knn_object.predict(embeddings)
        positive_predict = self.knn_positive.predict(embeddings)
        relevant_predict = self.knn_relevant.predict(embeddings)

        return {"object": object_predict, "positive": positive_predict, "relevant": relevant_predict}


In [6]:
data = pd.DataFrame({"key":[1, 2, 3], "is_relevant": [0, 0, 0], "object":[0, 0, 0], "is_positive": [0, 0, 0]})
data

,key,is_relevant,object,is_positive
0,1,0,0,0
1,2,0,0,0
2,3,0,0,0


In [7]:
data["is_positive"] = pd.Series([1, 2, 3])
data

,key,is_relevant,object,is_positive
0,1,0,0,1
1,2,0,0,2
2,3,0,0,3


In [8]:
data = json.loads(data.to_json(orient='records'))
data

[{'key': 1, 'is_relevant': 0, 'object': 0, 'is_positive': 1},
 {'key': 2, 'is_relevant': 0, 'object': 0, 'is_positive': 2},
 {'key': 3, 'is_relevant': 0, 'object': 0, 'is_positive': 3}]

## Загрузка тестового датасета

In [ ]:
data_test = pd.read_csv("")

data_json = json.loads(data_test.to_json(orient='records'))




In [36]:
object_cats = Model(
    model_object = "model_knn\\best_model_object.pkl", 
    model_positive = "model_knn\\best_model_positive.pkl", 
    model_relevant ="model_knn\\best_model_relevant.pkl")

d:\Hackathons\Hack_26_04_2024\.venv\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator KNeighborsClassifier from version 1.2.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [37]:
object_cats.predict([{"question_1":"Новейшие тенденции в IT","question_2":"Дискуссия о будущем облачных вычислений и их влиянии на ИТ-индустрию была очень проницательной.","question_3":"Нет.","question_4":"Включить больше кейс-стади от компаний, успешно интегрировавших облачные решения.","question_5":"Стратегии миграции предприятий в облако."}])

(array([2], dtype=int64), array([1], dtype=int64), array([1], dtype=int64))

In [32]:
object_cats.predict([{"question_1":"Новейшие тенденции в IT","question_2":"Дискуссия о будущем облачных вычислений и их влиянии на ИТ-индустрию была очень проницательной.","question_3":"Нет.","question_4":"Включить больше кейс-стади от компаний, успешно интегрировавших облачные решения.","question_5":"Стратегии миграции предприятий в облако."}])

(array([2], dtype=int64), array([1], dtype=int64), array([1], dtype=int64))

In [50]:
test = {"analysis_result": [
    {
      "user_id": 1229352473,
      "answers": {
        "1": "Разработка на Next JS",
        "2": "Разнообразие информации и подробное описание как работать с технологиями",
        "3": "Пока не было",
        "4": "Хотелось бы увидеть более удобный интерфейс для взаимодействия между сервисом и пользователем",
        "5": "Backend разработка на Node JS"
      }
    },
    {
      "user_id": 1321034543,
      "answers": {
        "1": "Победители Цифрового прорыва",
        "2": "Больше всего мне понравились сказочные истории, про мечтателей, которые захотели выиграть хакатон когда их фронт отвалился спать+работать",
        "3": "Максимально затруднительно работать, когда фронт лежит",
        "4": "Я считаю, что нам надо улучшить фронтендера, пусть затаривается энергосами - его ждет бессонная ночь",
        "5": "Я бы хотел изучить методы пинания фронтэндеров"
      }
    }
  ]}

In [51]:
[val['answers'] for val in test["analysis_result"]]

[{'1': 'Разработка на Next JS',
  '2': 'Разнообразие информации и подробное описание как работать с технологиями',
  '3': 'Пока не было',
  '4': 'Хотелось бы увидеть более удобный интерфейс для взаимодействия между сервисом и пользователем',
  '5': 'Backend разработка на Node JS'},
 {'1': 'Победители Цифрового прорыва',
  '2': 'Больше всего мне понравились сказочные истории, про мечтателей, которые захотели выиграть хакатон когда их фронт отвалился спать+работать',
  '3': 'Максимально затруднительно работать, когда фронт лежит',
  '4': 'Я считаю, что нам надо улучшить фронтендера, пусть затаривается энергосами - его ждет бессонная ночь',
  '5': 'Я бы хотел изучить методы пинания фронтэндеров'}]

In [52]:
test = {'1': 0.6963162660598755, '2': 0.3210538327693939, '0': 0.3826298415660858, }
max(test, key=test.get)

'1'